In [ ]:
!pip install face_recognition
!pip install ffmpeg
!pip install ffmpeg-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#Para a IA funcionar no colab corretamente é necessário uma GPU! Isto irá confirmar que uma GPU está ativa!
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9954790805968185888
 xla_global_id: -1, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14419755008
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 3569336994533595107
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [ ]:
import face_recognition
from moviepy.editor import *
import moviepy.editor
import ffmpeg
import glob
import cv2
import numpy as np
import os
import shutil
from tqdm.notebook import tqdm

In [ ]:
def censura_rosto(filename, top, right, bottom, left):
  
  #Lendo e borrando toda a imagem usando Convolussão Gaussiana - Blur
  image = cv2.imread(filename)
  blurred_image = cv2.GaussianBlur(image,(153, 153), 0)

  #Criando uma mask para delimitar a área borrada
  roi_corners = np.array([[(left,top),(right,top),(right,bottom),(left,bottom)]],dtype = np.int32)
  mask = np.zeros(image.shape, dtype=np.uint8)
  channel_count = image.shape[2]
  ignore_mask_color = (255,)*channel_count
  cv2.fillPoly(mask, roi_corners, ignore_mask_color)
  mask_inverse = np.ones(mask.shape).astype(np.uint8)*255 - mask

  #Juntando a mask e a imagem borrada
  final_image = cv2.bitwise_and(blurred_image, mask) + cv2.bitwise_and(image, mask_inverse)
  cv2.imwrite(filename,final_image)
  

In [ ]:
def censura_foto(filename, number_of_times_to_upsample = 2):

  image = face_recognition.load_image_file(filename)
  face_locations = face_recognition.face_locations(image, number_of_times_to_upsample)

  for (top, right, bottom, left) in face_locations: 
    censura_rosto(filename, top, right, bottom, left)
    

In [ ]:
def extrai_video(filename):

  #criando pasta para os frames
  
  os.makedirs(os.path.join("./", "video"))

  #extraindo o audio
  clip = moviepy.editor.VideoFileClip(filename)
  clip.audio.write_audiofile("./video/" +"audio.mp3")

  #extraindo os frames
  vidcap = cv2.VideoCapture(filename)
  success,image = vidcap.read()
  count = 0 

  while success:
    cv2.imwrite("./video/" + "frame%s.jpg" % str(count).zfill(8), image)     
    success,image = vidcap.read()
    count += 1

  print("Frames extraidos!")
  vidcap.release()
  cv2.destroyAllWindows()


In [ ]:
def censura_video(filename, number_of_times_to_upsample = 1):  
  #Atenção: Talvez seja possível fazer usando Batch para ser mais rápido na GPU, porém sempre falta memória!
  extrai_video(filename)

  files = glob.glob(f"/content/video/*.jpg")
  files.sort()

  for frames in tqdm(files, desc="Censurando os Frames... " ):
   censura_foto(frames, number_of_times_to_upsample)
  
  frame = cv2.imread(os.path.join("/content/video/", files[0]))
  height, width, layers = frame.shape
  video = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30, (width,height))

  for frame in tqdm(files, desc="Criando o vídeo... "):
    video.write(cv2.imread(os.path.join("./", frame)))
  
  cv2.destroyAllWindows()
  video.release()

  #juntando o audio do vídeo
  print("Criando o Audio... ")
  input_video = ffmpeg.input('./output.mp4')
  input_audio = ffmpeg.input('./video/audio.mp3')
  ffmpeg.concat(input_video, input_audio, v=1, a=1).output('./finished_video.mp4').run()

  #Apagando a pasta "./video/" e o "output.mp4" para evitar mesclagem de frames
  shutil.rmtree(os.path.join("./", "video"))
  os.remove("./output.mp4")